In [1]:
# read image data
import cv2
def read_image(path):
    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    return img

In [2]:
import json
def get_die_data(path):
    data = ""
    with open(path) as f:
        data = json.load(f)
    die_data,care_areas,exclusion_zones = data['die'],data['care_areas'],data['exclusion_zones']
    return die_data,care_areas,exclusion_zones

In [3]:
die_data,care_areas,exclusion_zones = get_die_data("/Users/shivshankar/Shiv/KLA_Hackathon/Level_1_data/input.json")

In [4]:
# check if pixel in care_area
def check_care_area_pixel(i,j,value):
    maxx_care = value['bottom_right']['x']
    miny_care = value['bottom_right']['y']
    minx_care = value['top_left']['x']
    maxy_care = value['top_left']['y']
    valid = 0
    if i >= minx_care and i <= maxx_care and j >= miny_care and j <= maxy_care:
        valid = 1
    return valid

In [5]:
# check if pixel in exclusion zones
def check_exclusion_zones_pixel(i,j,value):
    maxx_care = value['bottom_right']['x']
    miny_care = value['bottom_right']['y']
    minx_care = value['top_left']['x']
    maxy_care = value['top_left']['y']
    valid = 0
    if i >= minx_care and i <= maxx_care and j >= miny_care and j <= maxy_care:
        valid = 1
    return valid

In [6]:
# check weather count this error
def check_index(i,j,care_areas,exclusion_zones):
    is_in_care_area = 0
    is_in_exclusion_zones = 0
    for value in care_areas:
        is_in_care_area = is_in_care_area or check_care_area_pixel(i,j,value)
    for value in exclusion_zones:
        is_in_exclusion_zones = is_in_exclusion_zones or check_exclusion_zones_pixel(i,j,value)
    if is_in_exclusion_zones:
        return 0
    if is_in_care_area:
        return 1
    return 0

In [7]:
img = read_image("/Users/shivshankar/Shiv/KLA_Hackathon/Level_1_data/wafer_image_1.png")

In [8]:
# combine all image
import numpy as np
def combine_image():
    c = read_image("/Users/shivshankar/Shiv/KLA_Hackathon/Level_1_data/wafer_image_1.png")
    for i in range(2,6):
        img = read_image("/Users/shivshankar/Shiv/KLA_Hackathon/Level_1_data/wafer_image_"+str(i)+".png")
        c = np.concatenate((c,img),axis=1)
        print(c.shape)
    return c

In [9]:
# get all individual dies
import numpy as np
def individual_dies():
    c = combine_image()
    (j,i) = c.shape
    dies = dict()
    dies_count = 1
    for ind in range(0,i,die_data['width']):
        a = c[:,ind:ind+die_data['width']]
        dies[str(dies_count)] = a
        dies_count+=1
    return dies

In [10]:
# return optimal die

#calculate the frequency of pixel value at each coordinates
def calculate_freq():
    freq = []
    for i in range(die_data['height']):
        a = []
        for j in range(die_data['width']):
            s = dict()
            a.append(s)
        freq.append(a)

    all_dies = individual_dies()
    for [key,value] in all_dies.items():
        for i in range(die_data['height']):
            for j in range(die_data['width']):
                if str(value[i][j]) in freq[i][j]:
                    freq[i][j][str(value[i][j])]+=1
                else:
                    freq[i][j][str(value[i][j])]=1



    return freq

In [11]:
# return optimal die
def get_optimal_die():
    freq = calculate_freq()
    optimal_die = np.zeros((die_data['height'],die_data['width']))
    for i in range(die_data['height']):
        for j in range(die_data['width']):
            max_key = max(freq[i][j],key=freq[i][j].get)
            optimal_die[i][j] = max_key
            
    return optimal_die

In [12]:
# calculate error
# given all dies and optimal dies, calculate the error
def calculate_errors():
    optimal_die = get_optimal_die()
    all_dies = individual_dies()
    errors = []
    for [key,value] in all_dies.items():
        for i in range(die_data['height']):
            for j in range(die_data['width']):
                if value[i][j] != optimal_die[i][j] and check_index(i,j,care_areas,exclusion_zones):
                    error = []
                    error.append(key)
                    error.append(j)
                    error.append(die_data['height']-1-i)
                    errors.append(error)
    return errors

In [13]:
# write file to csv
import csv
def make_csv_file():
    errors = calculate_errors()
    with open('answer.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(errors)

In [14]:
make_csv_file()

(600, 1600)
(600, 2400)
(600, 3200)
(600, 4000)
(600, 1600)
(600, 2400)
(600, 3200)
(600, 4000)
